In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from matplotlib import pyplot as plt

tess_data_dir = f'/mnt/tess/lc'
reports_dir = f'/mnt/tess/rpt/png'
tces_file = '/mnt/tess/astronet/tces-v3.csv'

b = plt.get_cmap('tab20')(0)
b2 = plt.get_cmap('tab20')(1)
o = plt.get_cmap('tab20')(2)
o2 = plt.get_cmap('tab20')(3)
g = plt.get_cmap('tab20')(4)
r = plt.get_cmap('tab20')(6)
n = plt.get_cmap('tab20')(8)

fsize = (16, 9)
dpi = 250

tic = 21601170

In [ ]:
from astropy.io import fits
from light_curve_util import tess_io

file_names = tess_io.tess_filenames(tic, tess_data_dir)
f = fits.open(file_names)

td = f[1].data["TIME"]
fd = f[1].data["KSPSAP_FLUX"]
fs = f[1].data["SAP_FLUX"]

fig = plt.figure(figsize=fsize, dpi=dpi)
plt.plot(td, fd, '-', markersize=3, c=b, label='KSPSAP')
plt.plot(td, fs, 'o-', markersize=3, c=g, label='SAP')
plt.legend()
_ = plt.title('fits data')

In [ ]:
import pandas as pd


tce_table = pd.read_csv(tces_file, header=0)

tce = tce_table[tce_table.tic_id == tic]
period = tce.Period.values.item()
epoc = tce.Epoc.values.item()
duration = tce.Duration.values.item()

period, epoc, duration

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(4, 3), dpi=dpi)
plt.axis('off')
im = plt.imread(f'{reports_dir}/TIC{tic}.png')
_ = plt.imshow(im)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from astronet.preprocess import preprocess
from light_curve_util import keplersplinev2

t, f = preprocess.read_and_process_light_curve(tic, tess_data_dir, 'KSPSAP_FLUX')
ut, uf = preprocess.read_and_process_light_curve(tic, tess_data_dir, 'SAP_FLUX')

input_mask = preprocess.get_spline_mask(ut, period, epoc, duration)
sf, mdata = keplersplinev2.choosekeplersplinev2(
    ut, uf, input_mask=input_mask, verbose=True, return_metadata=True)
sf, mdata = keplersplinev2.choosekeplersplinev2(
    ut, uf, input_mask=None, verbose=True, return_metadata=True)

print(f'bkspace: {mdata.bkspace}')

fig = plt.figure(figsize=fsize, dpi=dpi)
plt.plot(t, f, '-', markersize=3, color=b, label='KSPSAP')
plt.plot(ut, uf, '-', markersize=3, alpha=0.6, color=g, label='SAP')
plt.plot(ut, sf, 'o', markersize=3, color=o, label='spline')
mask_plot = np.where(input_mask, 1.0, min(np.min(f), np.min(sf), np.min(uf)))
plt.plot(ut, mask_plot, '.', alpha=0.3, color=r, label='OOT')
plt.legend()
_ = plt.title('raw')

In [ ]:
ft, ff = preprocess.phase_fold_and_sort_light_curve(t, f, period, epoc)
fmt, fm = preprocess.phase_fold_and_sort_light_curve(t, input_mask, period, epoc)

ut, nf = preprocess.detrend_and_filter(tic, ut, uf, period, epoc, duration)
sft, sff = preprocess.phase_fold_and_sort_light_curve(ut, nf, period, epoc)

fig = plt.figure(figsize=fsize, dpi=dpi)
plt.plot(ft, ff, '-', markersize=3, c=b, label='original')
plt.plot(sft, sff, '-o', markersize=3, alpha=0.6, c=o, label='spline')
sff_filtered = np.where((sff > 1.5) | (sff < -0.5), 0, sff)
if len(sff_filtered):
    mask = np.where(fm, 1, min(sff_filtered))
    plt.plot(fmt, mask, 'o', markersize=3, alpha=0.6, c=r, label='OOT')
else:
    print('Filtering removed all data!')
plt.legend()
_ = plt.title('phase folded / filtered')

In [ ]:
gv = preprocess.global_view(tic, ft, ff, period)
sgv = preprocess.global_view(tic, sft, sff, period)

fig = plt.figure(figsize=fsize, dpi=dpi)
plt.plot(gv, '-', markersize=3, color=b, label='original')
plt.plot(sgv, 'o-', markersize=3, color=o, label='spline')
plt.legend()
_ = plt.title('global view')

In [ ]:
fig = plt.figure(figsize=fsize, dpi=dpi)

lv = preprocess.local_view(tic, ft, ff, period, duration, new_binning=False)
lvs = preprocess.local_view(tic, sft, sff, period, duration, new_binning=False)
plt.plot(lv, '-', markersize=3, c=b, label='original median')
plt.plot(lvs, 'o-', markersize=3, c=o, label='spline median')

lv = preprocess.local_view(tic, ft, ff, period, duration, new_binning=True)
lvs = preprocess.local_view(tic, sft, sff, period, duration, new_binning=True)
plt.plot(lv, '-', markersize=3, c=b2, label='original robust')
plt.plot(lvs, 'o-', markersize=3, c=o2, label='spline robust')

plt.legend()
_ = plt.title('local view')

In [ ]:
lv = preprocess.secondary_view(tic, ft, ff, period, duration)
lvs = preprocess.secondary_view(tic, sft, sff, period, duration)

fig = plt.figure(figsize=fsize, dpi=dpi)
plt.plot(lv, '-', markersize=3, color=b, label='original robust')
plt.plot(lvs, 'o-', markersize=3, color=o, label='spline robust')
plt.legend()
_ = plt.title('secondary view')

In [ ]:
from astronet.preprocess import generate_input_records


generate_input_records.FLAGS = generate_input_records.parser.parse_args([
  '--tess_data_dir', '/mnt/tess/lc',
  '--output_dir', '/dev/null',
  '--input_tce_csv_file', tces_file,
])


row = list(tce.iterrows())[0][1]

try:
    ex = generate_input_records._process_tce(row)
except:
    print('Using old detrending')
    ex = generate_input_records._process_tce(row, True)
    

fig = plt.figure(figsize=(18, 6), dpi=300)
plt.plot(ex.features.feature['global_view'].float_list.value, color=n)
plt.title('process_tce global')
fig = plt.figure(figsize=(18, 6), dpi=300)
plt.plot(ex.features.feature['local_view'].float_list.value, color=n)
plt.title('process_tce local')
fig = plt.figure(figsize=(18, 6), dpi=300)
plt.plot(ex.features.feature['secondary_view'].float_list.value, color=n)
_ = plt.title('process_tce secondary')